In [148]:
import pandas as pd
import numpy as np
import Schedule_Functions as sf
from datetime import datetime
from datetime import timedelta 

pd.set_option('max_rows', 20)

In [149]:
service_table = pd.read_csv("../Data/PAAC_Service_Table.csv")
service_table['route_no'] = service_table.route_no.apply(lambda x: str(x))

In [177]:
p68 = service_table[service_table.route_no == "P68"]
# p68.arr_time = p68.arr_time.apply(lambda x: datetime.strptime(x, '%I:%M %p') if type(x) == str else "")
# p68.dep_time = p68.dep_time.apply(lambda x: datetime.strptime(x, '%I:%M %p') if type(x) == str else "")
p68_copy = pd.DataFrame()
trips = list(p68.trip_id.unique())

# for t in trips:
#     temp = p68[p68.trip_id == t].sort_values("trip_stop_sequence").reset_index(drop = True)
    
#     time_thru = temp['arr_time'].iloc[temp.shape[0]-1] - \
#                  temp['arr_time'].iloc[0]
    
#     if time_thru.seconds/60 < 5:
#         continue
#     else:
#         p68_copy = p68_copy.append(temp)
        
# p68_copy = p68_copy.reset_index(drop = True)
# p68 = p68_copy

In [167]:
trips = list(p68.trip_id.unique())

mod_68 = pd.DataFrame()

for t in trips:
    temp = p68[p68.trip_id == t].sort_values("trip_stop_sequence").reset_index(drop = True)
    inbound = temp.direction_id.iloc[0] == "Inbound"
    
#     time_thru = temp['arr_time'].iloc[temp.shape[0]-1] - \
#                  temp['arr_time'].iloc[0]
    
#     if time_thru.seconds/60 <5:
#         continue
    
    if inbound:
        new_row = temp.iloc[0:1, :].copy()
        new_row.stop_id.iloc[0] = "x"
        new_row.stop_name.iloc[0] = "Dummy Stop"
        
        temp.arr_time = temp.arr_time.apply(lambda x: x + timedelta(minutes=20))
        temp.dep_time = temp.dep_time.apply(lambda x: x + timedelta(minutes=20))
        temp.trip_stop_sequence = temp.trip_stop_sequence.apply(lambda x: x+1)
        
        mod_68 = mod_68.append(new_row)
        mod_68 = mod_68.append(temp)
        
    else:
        new_row = temp[temp.trip_stop_sequence == max(temp.trip_stop_sequence)].copy()
        new_row.stop_id.iloc[0] = "x"
        new_row.stop_name.iloc[0] = "Dummy Stop"
        new_row.trip_stop_sequence.iloc[0] = new_row.trip_stop_sequence.iloc[0] + 1
        new_row.arr_time.iloc[0] = new_row.arr_time.iloc[0] + timedelta(minutes=20)
        new_row.dep_time.iloc[0] = new_row.dep_time.iloc[0] + timedelta(minutes=20)
        
        mod_68 = mod_68.append(temp)
        mod_68 = mod_68.append(new_row)

mod_68 = mod_68.reset_index(drop = True)

In [169]:
mod_68.arr_time = mod_68.arr_time.apply(lambda x: x if pd.isnull(x)  else
                                        datetime.strftime(x, '%I:%M %p'))
mod_68.dep_time = mod_68.dep_time.apply(lambda x: x if pd.isnull(x)  else
                                        datetime.strftime(x, '%I:%M %p'))

# mod_68.to_csv("../Data/Route Adjustments/modified_p68.csv", index=False)

## Checking Peak Vehicles Adjustments

In [198]:
def peak_vehicle(schedule, recovery_factor = 0.0):
    '''
    Given a schedule, returns the peak number of vehicles required to fulfill that schedule.
    Recovery factor is the percent of the trip that must be spent recovering.
    '''
    
    ### return "-" for empty dataframes
    if schedule.shape[0] == 0:
        return "-"
    
    ## Remove trips that have no departure time & arrival time information
    schedule = remove_complete_na_trips(schedule)
    
    ## Remove any trips where the time difference between first and last stops is less than 5 minutes
    trips = list(schedule.trip_id.unique())
    
    ## Convert arr_times to date type
    schedule.arr_time = schedule.arr_time.apply(lambda x: x if pd.isna(x) else datetime.strptime(x, '%I:%M %p'))

    new_df = pd.DataFrame()

    for t in trips:
        temp = schedule[schedule.trip_id == t].sort_values("trip_stop_sequence").reset_index(drop = True)
        
        first_stop_time = temp['arr_time'].iloc[0]
        
        last_stop_time_ix = temp.shape[0]-1
        
        if pd.isna(schedule.arr_time.iloc[last_stop_time_ix]):
                last_stop_time_ix = last_stop_time_ix-1
                
                while(pd.isna(schedule.arr_time.iloc[last_stop_time_ix])):
                    last_stop_time_ix = last_stop_time_ix-1
                    
                last_stop_time = temp['arr_time'].iloc[last_stop_time_ix]
                
        else:
            last_stop_time = temp['arr_time'].iloc[last_stop_time_ix]
        
        time_thru = last_stop_time - first_stop_time

        if time_thru.seconds/60 <= 25:
            continue
        else:
            new_df = new_df.append(temp)
            
    schedule = new_df.reset_index(drop = True)
    
    ## First order schedule by trips and stops
    schedule = schedule.sort_values(["trip_id", "trip_stop_sequence"]).reset_index(drop = True)
    
    ## Second go through all trips to get two lists. One with all departure times from first stops
    ## and second with (adjusted by recovery factor) arrival times for all last stops on trip.
    
    departure_times = list()
    arrival_times = list()

    for i in range(schedule.shape[0]):

        if i == 0:
            departure_times.append(schedule.arr_time[i])

        elif i == (schedule.shape[0]-1):

            if pd.isna(schedule.arr_time[i]):
                j = i-1
                while(pd.isna(schedule.arr_time[j])):
                    j = j-1
                arrival_times.append(schedule.arr_time[j])
            else:
                arrival_times.append(schedule.arr_time[i])

        elif schedule.trip_stop_sequence[i] == 1:

            # Some cases where we have na in columns
            if pd.isna(schedule.arr_time[i-1]):
                j = i-1
                while(pd.isna(schedule.arr_time[j])):
                    j = j-1
                arrival_times.append(schedule.arr_time[j])
            else:
                arrival_times.append(schedule.arr_time[i-1])

            departure_times.append(schedule.arr_time[i])

    # Accouting for recovery factor, we adjust arrival times
    adjusted_arrivals = [arrival_times[i] + ((arrival_times[i] - departure_times[i]) * (recovery_factor)) \
                         for i in range(len(arrival_times))]
    
    ## Third, create a dataframe that has all first and last stop times.
    trips_df = pd.DataFrame(zip(departure_times, adjusted_arrivals), columns = ["Departure", "Arrival"])

    # Assume that any trip that departs (or arrives) between the hours of 12am and 3am occurs on the next day
    for i in range(0, trips_df.shape[0]):
    
        if trips_df.Departure[i].hour <= 3:
            trips_df.loc[i, "Departure"] = trips_df.loc[i, "Departure"] + timedelta(days=1)
            trips_df.loc[i, "Arrival"] = trips_df.loc[i, "Arrival"] + timedelta(days=1)
        elif trips_df.Arrival[i].hour <= 3:
            trips_df.loc[i, "Arrival"] = trips_df.loc[i, "Arrival"] + timedelta(days=1)

    trips_df = trips_df.sort_values("Departure").reset_index(drop=True)
    
    ## Finally  count how many busses overlap at any one time and record the maximums
    peak_vehicles = 0

    for i in range(trips_df.shape[0]):

        if i != trips_df.shape[0]-1:
            inservice_vehicles = 0
            temp_arrival = trips_df.Arrival[i]

            j=i

            while trips_df.Departure[j] < temp_arrival:
                inservice_vehicles += 1
                j+=1

                if j == trips_df.shape[0]-1: break

            if inservice_vehicles > peak_vehicles:
                peak_vehicles = inservice_vehicles
                
    return peak_vehicles

In [200]:
peak_vehicle(mod_68)

6

In [201]:
#p68.arr_time = p68.arr_time.apply(lambda x: x if pd.isnull(x) else datetime.strftime(x, '%I:%M %p'))
peak_vehicle(p68)

5

## Exporting original to easy manipulated schedule format

In [66]:
ez_sched1

,Trip,Beginning,End,Direction
0,11022-1579400,05:39 AM,06:38 AM,Inbound
1,11023-1579403,06:14 AM,07:13 AM,Inbound
2,11024-1579397,06:44 AM,07:43 AM,Inbound
3,11025-1579262,07:13 AM,08:13 AM,Inbound
4,11026-1579400,07:48 AM,08:48 AM,Inbound
5,11028-1579404,08:18 AM,09:18 AM,Inbound
6,11029-1579262,09:30 AM,10:26 AM,Inbound
7,11030-1579404,10:30 AM,11:26 AM,Inbound
8,11031-1579542,11:30 AM,12:26 PM,Inbound
9,11032-1579267,12:30 PM,01:27 PM,Inbound


In [63]:
ez_sched1 = pd.DataFrame()

trips = list(p68.trip_id.unique())

for t in trips:
    temp = p68[p68.trip_id == t].sort_values("trip_stop_sequence").reset_index(drop = True)
    
    begin = temp.arr_time.iloc[0]
    end = temp[temp.trip_stop_sequence == max(temp.trip_stop_sequence)].arr_time.iloc[0]
    direction = temp.direction_id.iloc[0]
    
    if (not pd.isnull(begin)) & (not pd.isnull(end)):
        ez_sched1 = ez_sched1.append(pd.DataFrame(data = zip([t], [begin], [end], [direction]),
                                                 columns = ["Trip", "Beginning", "End", "Direction"]))
        
ez_sched1.Beginning = ez_sched1.Beginning.apply(lambda x: datetime.strftime(x, '%I:%M %p'))
ez_sched1.End = ez_sched1.End.apply(lambda x: datetime.strftime(x, '%I:%M %p'))
ez_sched1 = ez_sched1.reset_index(drop = True)

## Exporting manipulated to easy manipulated schedule format

In [11]:
ez_sched2 = pd.DataFrame()

mod_68_wd = mod_68[mod_68.wkdy_service == "Yes"]
trips = list(mod_68_wd.trip_id.unique())

for t in trips:
    temp = mod_68_wd[mod_68_wd.trip_id == t].sort_values("trip_stop_sequence").reset_index(drop = True)
    
    begin = temp.arr_time.iloc[0]
    end = temp[temp.trip_stop_sequence == max(temp.trip_stop_sequence)].arr_time.iloc[0]
    direction = temp.direction_id.iloc[0]
    
    if (not pd.isnull(begin)) & (not pd.isnull(end)):
        ez_sched2 = ez_sched2.append(pd.DataFrame(data = zip([t], [begin], [end], [direction]),
                                                 columns = ["Trip", "Beginning", "End", "Direction"]))
        
# ez_sched2.Beginning = ez_sched2.Beginning.apply(lambda x: datetime.strftime(x, '%I:%M %p'))
# ez_sched2.End = ez_sched2.End.apply(lambda x: datetime.strftime(x, '%I:%M %p'))
ez_sched2 = ez_sched2.reset_index(drop = True)

In [18]:
# Writing to excel
# ez_sched.to_csv("../Data/Route Adjustments/p68_ez.csv", index = False)
writer = pd.ExcelWriter('../Data/Route Adjustments/p68_ez.xlsx', engine='xlsxwriter')
ez_sched1.to_excel(writer, sheet_name='Original')
ez_sched2.to_excel(writer, sheet_name='Modified')
writer.save()